<a href="https://colab.research.google.com/github/uni2237/Data_Analysis/blob/master/%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini Project - 수입 우범화물 선별

# 1. 분석 환경 설정 및 데이터 로딩

In [1]:
# 모델 구현 시간 체크
import time

# 시작시간 체크
start = time.time()

# 패키지 로딩
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 한글폰트 적용
plt.rcParams['font.family'] = 'Malgun Gothic'

# 수입 데이터 로딩 (가상데이터)
train_xlsx = pd.read_excel("train.xlsx")
train_xlsx.to_csv("train.csv")
df_org = pd.read_csv('train.csv',index_col=[0])

test_xlsx = pd.read_excel("test.xlsx")
test_xlsx.to_csv("test.csv")
df_org_ts=pd.read_csv('test.csv',index_col=[0]) #unnamed 삭제 후 로드

# 2. 데이터 전처리

### 2.1 원산-적출 컬럼 생성 함수

In [2]:
# 원산지-적출 컬럼 생성
df_org["원산지적출"] = df_org["원산지국가코드"] +df_org["적출국가코드"]
df_org_ts["원산지적출"] = df_org_ts["원산지국가코드"] +df_org_ts["적출국가코드"]


# 원산지 / 적출 / 원산지적출 리스트 생성
def make_list(df_org):
    result_1=df_org['원산지국가코드'][df_org['우범여부']==1].value_counts()
    result_0=df_org['원산지국가코드'][df_org['우범여부']==0].value_counts()

    result_11=df_org['적출국가코드'][df_org['우범여부']==1].value_counts()
    result_01=df_org['적출국가코드'][df_org['우범여부']==0].value_counts()


    result_all = df_org['원산지국가코드'].value_counts()
    result_all1 = df_org['적출국가코드'].value_counts()

  
    result_12=df_org['원산지적출'][df_org['우범여부']==1].value_counts()
    result_02=df_org['원산지적출'][df_org['우범여부']==0].value_counts()
    result_all2 = df_org['원산지적출'].value_counts()

    a_b=[]
    a=[]
    b=[]
    for i in result_1.keys():
      #print(i,"의 우범비율 %.2f%%" % (result_1[i] / result_all[i]* 100.0), "   (",result_all[i],"개 중 ",result_1[i],")")
      if int(result_1[i] / result_all[i]* 100.0)>=50:
        a.append(i)

    for i in result_11.keys():
        #print(i,"의 우범비율 %.2f%%" % (result_11[i] / result_all1[i]* 100.0), "   (",result_all1[i],"개 중 ",result_11[i],")")
        if int(result_11[i] / result_all1[i]* 100.0)>=50:
          b.append(i)
    for i in sorted(result_12.keys()):
        #print(i,"의 우범비율 %.2f%%" % (result_12[i] / result_all2[i]* 100.0), "   (",result_all2[i],"개 중 ",result_12[i],")")
        if int(result_12[i] / result_all2[i]* 100.0)>=70:
          a_b.append(i)
    print("a",a)
    print("b",b)
    print("a_b",a_b)
    print(len(a_b))

    #신고인 부호 리스트 생성
    big_list=[]  # 1로 바꿔줄 신고인부호 저장할 리스트 (우범 횟수가 100이상 또는 우범 비열이 50% 이상인 신고인들을 1로 둠)
    result_1=df_org['신고인부호'][df_org['우범여부']==1].value_counts()
    result_all=df_org['신고인부호'].value_counts()
    count=0
    for i in result_1:
        if i>=100:
            count+=1
        else:
            break

    for i in result_1.keys()[0:count]:
        big_list.append(i)

    for i in sorted(result_1.keys()):
        if (result_1[i] / result_all[i]) * 100 >=50:
            big_list.append(i)

    return a,b,a_b,big_list

### 2.2 원산지-적출 생성

In [3]:
a,b,a_b,big_list=make_list(df_org)

a ['BA', 'UA', 'HR', 'AM', 'BO', 'GT']
b ['IE', 'AR', 'LI']
a_b ['AMIT', 'ARAR', 'ATTW', 'BADE', 'CNAT', 'CNTR', 'ESSG', 'GBJP', 'GBNL', 'GTNL', 'HRVN', 'INMY', 'JPPL', 'KRMX', 'PTDE', 'THGB', 'THVN', 'TRNL']
18


### 2.3 이 후 전처리 기능을 하는 함수

In [4]:
def data_process(df_org):
    
    
    df_org = pd.get_dummies(df_org, columns = ['통관지세관부호', '징수형태코드', '운송수단유형코드'])
    
  
  # 원산지 / 적출 / 원산지 적출

    def function_a(row):
        if row['원산지국가코드'] in a:
            return 1
        else:
            return 0
    def function_b(row):
        if row['적출국가코드'] in b:
            return 1
        else:
            return 0
    def function_a_b(row):
        if row['원산지적출'] in a_b:
            return 1
        else:
            return 0
    df_org['원산지국가코드'] =df_org.apply(function_a,axis=1)
    df_org['적출국가코드'] =df_org.apply(function_b,axis=1)
    df_org['원산지적출'] =df_org.apply(function_a_b,axis=1)


  # 수치형 변수를 자연로그로 변환(13, 14)
    for var in ['신고중량(KG)', '과세가격원화금액']:
        df_org[var] = df_org[var].apply(lambda x: np.log1p(x))

    
  # 신고 날짜 전처리를 위해 함수 정의
  #  def extract_month(row): 
  #      if int(row.split('-')[1]) <= 6:
  #          return 0
  #      else:
  #          return 1
  #  def extract_day(row) : return int(row.split('-')[2])//10
    def extract_day(row) : return int(row.split('-')[2])//10


#   # 2.신고 일자
#     df_org['월'] = df_org['신고일자'].apply(extract_month)
    df_org['일'] = df_org['신고일자'].apply(extract_day)
    del df_org['신고일자']
    df_org = pd.get_dummies(df_org, columns = ['일'])

    
  # 4.신고인부호
    def function_man(row):
        if row['신고인부호'] in big_list:
            return 1
        else:
            return 0
    df_org['신고인부호'] =df_org.apply(function_man,axis=1)


    # 6.해외거래처부호
    df = df_org['해외거래처부호'].fillna('missing') # nan을 missing으로 치환
    df_org['해외거래처부호'] = df 
    df_org=pd.get_dummies(df_org, columns = ['해외거래처부호'])
    missing = df_org.filter(regex='^해외거래처부호_missing') 
    df_org.drop(df_org.filter(regex='^해외거래처부호_'),axis=1,inplace=True) #missing 컬럼 제외하고 다지우기
    df_org['해외거래처부호_missing'] =  missing


    # 7.특송업체번호
    df = df_org['특송업체부호'].fillna('missing') # nan을 missing으로 치환
    df_org['특송업체부호'] = df 
    df_org=pd.get_dummies(df_org, columns = ['특송업체부호'])
    missing = df_org.filter(regex='^특송업체부호_missing') 
    df_org.drop(df_org.filter(regex='^특송업체부호_'),axis=1,inplace=True) #missing 컬럼 제외하고 다지우기
    df_org['특송업체부호_missing'] =  missing


    # 8.수입통관계획코드
    df_org=pd.get_dummies(df_org, columns = ['수입통관계획코드'])
    df_org['수입통관계획코드AG'] = 0
    df_org["수입통관계획코드CD"] = df_org["수입통관계획코드_C"] +df_org["수입통관계획코드_D"]
    df_org["수입통관계획코드EF"] = df_org["수입통관계획코드_E"] +df_org["수입통관계획코드_F"]
    df_org["수입통관계획코드HZ"] = df_org["수입통관계획코드_H"] +df_org["수입통관계획코드_Z"]
    df_org.drop(df_org.filter(regex='^수입통관계획코드_'),axis=1,inplace=True)


    # 9.수입신고구분코드
    df_org=pd.get_dummies(df_org, columns = ['수입신고구분코드'])
    df_org['수입신고구분코드CFG'] = 0


    # 10.수입거래구분코드
    df_org['수입거래구분코드_십'] = df_org['수입거래구분코드'] // 10
    df_org['수입거래구분코드_일'] = df_org['수입거래구분코드'] % 10
    del df_org['수입거래구분코드']
    df_org = pd.get_dummies(df_org, columns = ['수입거래구분코드_십', '수입거래구분코드_일'])


    # 11.수입종류코드(
    df_org.loc[df_org['수입종류코드'] == 21, '수입종류코드'] = 0
    df_org.loc[df_org['수입종류코드'] == 11, '수입종류코드'] = 0
    df_org.loc[df_org['수입종류코드'] != 0, '수입종류코드'] = 1
    df_org = pd.get_dummies(df_org, columns = ['수입종류코드'])


    # 16.반입보세구역부호
    def extract_1(row): return str(row)[0]
    def extract_2(row): return str(row)[1]
    def extract_3(row): return str(row)[0]
    def extract_4(row): return str(row)[1]
    df_org['반입보세구역부호_1'] = df_org['반입보세구역부호'].apply(extract_1)
    df_org['반입보세구역부호_2'] = df_org['반입보세구역부호'].apply(extract_2)
    df_org['반입보세구역부호_3'] = df_org['반입보세구역부호'].apply(extract_3)
    df_org['반입보세구역부호_4'] = df_org['반입보세구역부호'].apply(extract_4)
    df_org = pd.get_dummies(df_org, columns = ['반입보세구역부호_1', '반입보세구역부호_2', '반입보세구역부호_3', '반입보세구역부호_4'])
    del df_org['반입보세구역부호']

    
    # 17-1. HS10단위 부호(주요 코드)
    df_hs=df_org[['HS10단위부호']]/100000000
    df_hs=df_hs.astype(int)
    df_org['HS10단위']=df_hs
     
    #아래 주석에서 만들어진 hs_list가 주석 하단의 main_hs 입니다.
    #result_h1=df_org['HS10단위'][df_org['우범여부']==1].value_counts()
    #result_h0=df_org['HS10단위'][df_org['우범여부']==0].value_counts()
    #result_allh = df_org['HS10단위'].value_counts()
    #hs_list=[]
    #for i in sorted(result_h1.keys()):
    #    print(i,"의 우범비율 %.2f%%" % (result_h1[i] / result_allh[i]* 100.0), "   (",result_allh[i],"개 중 ",result_h1[i],")")
    #    if int(result_h1[i] / result_allh[i]* 100.0)>=50:
    #        hs_list.append(i)
    
    
    main_hs=[36, 78, 14, 93, 79]
    def function_hs(row):
        if row['HS10단위'] in main_hs:
            return 1
        else:
            return 0
    df_org['HS10단위'] =df_org.apply(function_hs,axis=1)
    
    
    # 17-2.HS10단위부호
    df_org['HS10단위부호_1'] = df_org['HS10단위부호'].apply(extract_1)
    df_org['HS10단위부호_2'] = df_org['HS10단위부호'].apply(extract_2)
    df_org = pd.get_dummies(df_org, columns = ['HS10단위부호_1', 'HS10단위부호_2'])
    del df_org['HS10단위부호']
    


    # 20.관세율 구분 코드
    df_org=pd.get_dummies(df_org, columns = ['관세율구분코드']) # 관세율구분코드를 원핫인코딩으로 변환
    df_org["관세율구분_A"]=df_org.iloc[:,-35]   # 각 범위로 그룹화,
    df_org["관세율구분_CE"]=df_org.iloc[:,-35:-30].sum(axis=1)
    df_org["관세율구분_FC"]=df_org.iloc[:,-31:-22].sum(axis=1)
    df_org["관세율구분_FE"]=df_org.iloc[:,-23:-9].sum(axis=1)
    df_org["관세율구분_LP"]=df_org.iloc[:,-10:-7].sum(axis=1)
    df_org["관세율구분_RW"]=df_org.iloc[:,-8:-5].sum(axis=1)
    df_org.drop(df_org.filter(regex='^관세율구분코드_'),axis=1,inplace=True)

    # 1 2 3 4 [5] 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 [21] 22 23

  

    del df_org['수입자부호']
    del df_org['관세율']
    return df_org

### 2.4 전처리 결과 저장 및 불필요한 변수 삭제 (trian.csv)

In [5]:
# 불필요한 변수 삭제(1, 22, 23)
del df_org['신고번호']
del df_org['검사결과코드']

df_org_train=data_process(df_org)

### 2.5 전처리 결과 저장 및 불필요한 변수 삭제 (test.csv)

In [6]:
del df_org_ts['신고번호']
df_org_ts=data_process(df_org_ts)
test_data = df_org_ts

print("train_data shape :" ,df_org_train.shape)
print("test_data shape :", test_data.shape)

train_data shape : (76837, 158)
test_data shape : (23163, 156)


### 2.6 Class-imbalanced 해결 : SMOTE

In [7]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=0)

org_train_y = df_org_train.pop('우범여부')
x,y = smote.fit_resample(df_org_train, org_train_y)
core_y = x.pop('핵심적발')
print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', df_org_train.shape, org_train_y.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', x.shape, y.shape)
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y).value_counts())

SMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (76837, 157) (76837,)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (118530, 156) (118530,)
SMOTE 적용 후 레이블 값 분포: 
 1    59265
0    59265
Name: 우범여부, dtype: int64


# 3. 모델 생성 및 훈련

### 3.1 모델, K-Fold, Score(acc, recall, precision, f1) Import

In [8]:
# 모델 import
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()

# K-Fold
from sklearn.model_selection import KFold
kf=KFold(n_splits=5, shuffle=True)

# Acc, F1 score
from sklearn.metrics import roc_auc_score, recall_score, precision_score, f1_score

# warnings
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm

### 3.2 K-Fold로 Split

### 3.3 score를 저장할 dict

In [9]:
score = {
    'RF' :{
        'accuracy' : [], 'precision' : [], 'recall' : [], 'f1_score' : [],
    },
    'XGBOOST' :{
        'accuracy' : [], 'precision' : [], 'recall' : [], 'f1_score' : [],
    },
    'GBM' :{
        'accuracy' : [], 'precision' : [], 'recall' : [], 'f1_score' : [],
    },
    'DNN' :{
        'accuracy' : [], 'precision' : [], 'recall' : [], 'f1_score' : [],
    },
}


### 3.4 모델 생성

In [10]:
def CreateModel(length):
    
    input_shape = (length,)
    mlp_model = models.Sequential()
    mlp_model.add(layers.Dense(units = 10, activation = 'relu', input_shape=input_shape))
    mlp_model.add(layers.Dense(units = 20, activation = 'relu'))
    mlp_model.add(layers.Dense(units = 20, activation = 'relu'))
    mlp_model.add(layers.Dense(units = 2, activation = 'softmax'))
    mlp_model.compile(optimizer='Adam', loss = 'binary_crossentropy', metrics=['accuracy'])
    return mlp_model

RF=RandomForestClassifier(max_depth=22, n_estimators=300, random_state=0,n_jobs=-1)
XGBOOST= XGBClassifier(n_estimators=10, max_depth=4)
GBM = GradientBoostingClassifier(max_depth=3, n_estimators=30, random_state=0)
#DNN = CreateModel(df_org_train.shape[1])

### 3.5 학습 및 Score 저장

In [11]:
# 학습, 예측
for train_index, test_index in tqdm(kf.split(x, y)):
    x_train, x_test, y_train, y_test=\
    x.iloc[train_index], x.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    
    RF.fit(x_train, y_train.ravel())
    XGBOOST.fit(x_train, y_train.ravel(), eval_metric=["logloss"], verbose=True)
    GBM.fit(x_train, y_train.ravel())
#     DNN.fit(x_train, encoder.fit_transform(y_train), batch_size=10, epochs=50)
    
    score['RF']['accuracy'].append(RF.score(x_test, y_test.values.ravel()))
    score['RF']['precision'].append(precision_score(RF.predict(x_test), y_test))
    score['RF']['recall'].append(recall_score(RF.predict(x_test), y_test))
    score['RF']['f1_score'].append(f1_score(RF.predict(x_test), y_test))
    
    score['XGBOOST']['accuracy'].append(XGBOOST.score(x_test, y_test.values.ravel()))
    score['XGBOOST']['precision'].append(precision_score(XGBOOST.predict(x_test), y_test))
    score['XGBOOST']['recall'].append(recall_score(XGBOOST.predict(x_test), y_test))
    score['XGBOOST']['f1_score'].append(f1_score(XGBOOST.predict(x_test), y_test))
    
    score['GBM']['accuracy'].append(GBM.score(x_test, y_test.values.ravel()))
    score['GBM']['precision'].append(precision_score(GBM.predict(x_test), y_test))
    score['GBM']['recall'].append(recall_score(GBM.predict(x_test), y_test))
    score['GBM']['f1_score'].append(f1_score(GBM.predict(x_test), y_test))
    
#     score['DNN']['accuracy'].append(DNN.score(x_test, y_test.values.ravel()))
#     score['DNN']['precision'].append(precision_score(DNN.predict(x), y))
#     score['DNN']['recall'].append(recall_score(DNN.predict(x), y))
#     score['DNN']['f1_score'].append(f1_score(DNN.predict(x), y))

    
# Score 출력
print(f'1-1. RandomForestClassifier [평균 Accuracy] :', np.mean(score['RF']['accuracy'])*100 , ',  [Accuracy 편차] :', np.std(score['RF']['accuracy']))
print(f'1-2. RandomForestClassifier [평균 Precision] :', np.mean(score['RF']['precision'])*100 , ', [Precision 편차]:', np.std(score['RF']['precision']))
print(f'1-3. RandomForestClassifier [평균 Recall] :', np.mean(score['RF']['recall'])*100 , ',    [Recall 편차]   :', np.std(score['RF']['recall']))
print(f'1-4. RandomForestClassifier [평균 F1_score] :', np.mean(score['RF']['f1_score'])*100 , ',   [F1_score 편차] :', np.std(score['RF']['f1_score']),'\n')

print(f'2-1. XGBOOSTClassifier [평균 Accuracy] :', np.mean(score['XGBOOST']['accuracy'])*100 , ',  [Accuracy 편차] :', np.std(score['XGBOOST']['accuracy']))
print(f'2-2. XGBOOSTClassifier [평균 Precision] :', np.mean(score['XGBOOST']['precision'])*100 , ', [Precision 편차] :', np.std(score['XGBOOST']['precision']))
print(f'2-3. XGBOOSTClassifier [평균 Recall] :', np.mean(score['XGBOOST']['recall'])*100 , ',   [Recall 편차] :', np.std(score['XGBOOST']['recall']))
print(f'2-4. XGBOOSTClassifier [평균 F1_score] :', np.mean(score['XGBOOST']['f1_score'])*100 , ',  [F1_score 편차] :', np.std(score['XGBOOST']['f1_score']),'\n')

print(f'3-1. GradientBoostingClassifier [평균 Accuracy] :', np.mean(score['GBM']['accuracy'])*100 , ',  [Accuracy 편차] :', np.std(score['GBM']['accuracy']))
print(f'3-2. GradientBoostingClassifier [평균 Precision] :', np.mean(score['GBM']['precision'])*100 , ', [Precision 편차] :', np.std(score['GBM']['precision']))
print(f'3-3. GradientBoostingClassifier [평균 Recall]   :', np.mean(score['GBM']['recall'])*100 , ',    [Recall 편차] :', np.std(score['GBM']['recall']))
print(f'3-4. GradientBoostingClassifier [평균 F1_score] :', np.mean(score['GBM']['f1_score'])*100 , ',   [F1_score 편차] :', np.std(score['GBM']['f1_score']))
                                                                                                                          
# print(f'1-2. DeepNeuralNetworkClassifier [평균 Accuracy] :', np.mean(score['DNN']['accuracy'])*100 , ',  [Accuracy 편차] :', np.std(score['DNN']['accuracy']))
# print(f'1-2. DeepNeuralNetworkClassifier [평균 Precision] :', np.mean(score['DNN']['precision'])*100 , ', [Precision 편차] :', np.std(score['DNN']['precision']))
# print(f'1-2. DeepNeuralNetworkClassifier [평균 Recall]   :', np.mean(score['DNN']['recall'])*100 , ',    [Recall 편차] :', np.std(score['DNN']['recall']))
# print(f'1-2. DeepNeuralNetworkClassifier [평균 F1_score] :', np.mean(score['DNN']['f1_score'])*100 , ',   [F1_score 편차] :', np.std(score['DNN']['f1_score']))

5it [06:18, 75.64s/it]

1-1. RandomForestClassifier [평균 Accuracy] : 85.54036952670211 ,  [Accuracy 편차] : 0.0027103163228880357
1-2. RandomForestClassifier [평균 Precision] : 88.05180874471581 , [Precision 편차]: 0.0039910347859484704
1-3. RandomForestClassifier [평균 Recall] : 83.84050740154191 ,    [Recall 편차]   : 0.003353785132827144
1-4. RandomForestClassifier [평균 F1_score] : 85.89399618319369 ,   [F1_score 편차] : 0.0029179670045714826 

2-1. XGBOOSTClassifier [평균 Accuracy] : 80.25394414916053 ,  [Accuracy 편차] : 0.0037068637094094512
2-2. XGBOOSTClassifier [평균 Precision] : 82.90629591280985 , [Precision 편차] : 0.005977888972653222
2-3. XGBOOSTClassifier [평균 Recall] : 78.73079346294915 ,   [Recall 편차] : 0.0036349968336406866
2-4. XGBOOSTClassifier [평균 F1_score] : 80.76348134365875 ,  [F1_score 편차] : 0.003810183003523968 

3-1. GradientBoostingClassifier [평균 Accuracy] : 79.34953176411035 ,  [Accuracy 편차] : 0.0028358550737391736
3-2. GradientBoostingClassifier [평균 Precision] : 81.32917544482127 , [Precision 편차] : 0.0

### 3.6 핵심적발을 예측하는 모델의 Score를 저장할 dict 생성

In [12]:
score_2 = {
    'RF' :{
        'accuracy' : [], 'precision' : [], 'recall' : [], 'f1_score' : [],
    },
    'XGBOOST' :{
        'accuracy' : [], 'precision' : [], 'recall' : [], 'f1_score' : [],
    },
    'GBM' :{
        'accuracy' : [], 'precision' : [], 'recall' : [], 'f1_score' : [],
    },
    'DNN' :{
        'accuracy' : [], 'precision' : [], 'recall' : [], 'f1_score' : [],
    },
}

### 3.7 모델 생성 및 학습, Score 저장

In [13]:
# Model 생성
RF_2=RandomForestClassifier(max_depth=22, n_estimators=300, random_state=0,n_jobs=-1)

x['우범여부'] = y
y=core_y

# 학습, 예측
for train_index, test_index in tqdm(kf.split(x, y)):
    x_train, x_test, y_train, y_test=\
    x.iloc[train_index], x.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    RF_2.fit(x_train, y_train.ravel())
    score_2['RF']['accuracy'].append(RF_2.score(x_test, y_test.values.ravel()))
    score_2['RF']['precision'].append(precision_score(RF_2.predict(x_test), y_test, average='micro'))
    score_2['RF']['recall'].append(recall_score(RF_2.predict(x_test), y_test, average='micro'))
    score_2['RF']['f1_score'].append(f1_score(RF_2.predict(x_test), y_test, average='micro'))
    
print(f'1-1. RandomForestClassifier [평균 Accuracy] :', np.mean(score_2['RF']['accuracy'])*100 , ',  [Accuracy 편차] :', np.std(score_2['RF']['accuracy']))
print(f'1-2. RandomForestClassifier [평균 Precision] :', np.mean(score_2['RF']['precision'])*100 , ', [Precision 편차]:', np.std(score_2['RF']['precision']))
print(f'1-3. RandomForestClassifier [평균 Recall] :', np.mean(score_2['RF']['recall'])*100 , ',    [Recall 편차]   :', np.std(score_2['RF']['recall']))
print(f'1-4. RandomForestClassifier [평균 F1_score] :', np.mean(score_2['RF']['f1_score'])*100 , ',   [F1_score 편차] :', np.std(score_2['RF']['f1_score']),'\n')

5it [03:38, 43.78s/it]

1-1. RandomForestClassifier [평균 Accuracy] : 86.6422002868472 ,  [Accuracy 편차] : 0.0030766257974661907
1-2. RandomForestClassifier [평균 Precision] : 86.6422002868472 , [Precision 편차]: 0.0030766257974661907
1-3. RandomForestClassifier [평균 Recall] : 86.6422002868472 ,    [Recall 편차]   : 0.0030766257974661907
1-4. RandomForestClassifier [평균 F1_score] : 86.6422002868472 ,   [F1_score 편차] : 0.00307662579746615 



### 3.8 Test data Load, Predict

In [14]:
test_xlsx = pd.read_excel("test.xlsx")
test_xlsx.to_csv("test.csv")
df_org_ts=pd.read_csv('test.csv',index_col=[0]) #unnamed 삭제 후 로드

In [15]:
import openpyxl

# test data preprocessing
df_org_ts["원산지적출"] = df_org_ts["원산지국가코드"] + df_org_ts["적출국가코드"]
df_org_ts["원산지적출"] = df_org_ts["원산지국가코드"] + df_org_ts["적출국가코드"]
num = df_org_ts.pop('신고번호')
df_org_ts = data_process(df_org_ts)

# predict
result_1 = RF.predict(df_org_ts)
df_org_ts['우범여부']= result_1
result_2 = RF_2.predict(df_org_ts)

submission = pd.DataFrame({
    "신고번호" : num,
    "우범여부" : result_1,
    "핵심적발" : result_2
})
submission.to_excel('RF_submission.xlsx', index=False)